# this is to scan Dec, on double -low Dx case

In [10]:
# magic lines that avoid re-start
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import pybamm;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp,sqrt;
import matplotlib as mpl; 
fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
import openpyxl
import traceback
import multiprocessing
font = {'family' : 'Times New Roman','size': fs}
mpl.rc('font', **font)

In [12]:
import sys  
str_path_0 = os.path.abspath(os.path.join(pybamm.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip\Rio_Code\Fun_P3"))
sys.path.append(str_path_1) 
from Fun_P3 import *
BasicPath = "D:/OneDrive - Imperial College London/SimDataSave/P3R13/Paper/"  #'D:/OneDrive - Imperial College London/SimDataSave/P3R13/'; 
Target  = 'Scan_Dec_double_low_Dx/' 
if not os.path.exists(BasicPath + Target):
   os.mkdir(BasicPath + Target)

In [13]:
def EC_diffusivity_5E_10(c_e, c_EC , T):
    D_ec_dim = (
        (c_EC >= 0 ) * 5e-10
        +  (c_EC < 0 ) * 0 
    )
    return D_ec_dim
def EC_diffusivity_3E_10(c_e, c_EC , T):
    D_ec_dim = (
        (c_EC >= 0 ) * 3e-10
        +  (c_EC < 0 ) * 0 
    )
    return D_ec_dim
def EC_diffusivity_1E_10(c_e, c_EC , T):
    D_ec_dim = (
        (c_EC >= 0 ) * 1e-10
        +  (c_EC < 0 ) * 0 
    )
    return D_ec_dim
def EC_diffusivity_5E_11(c_e, c_EC , T):
    D_ec_dim = (
        (c_EC >= 0 ) * 5e-11
        +  (c_EC < 0 ) * 0 
    )
    return D_ec_dim

In [14]:
# define parameter of double and single 
Para_dict_Same = {
   ################################# Same for single and double:
   "Mesh list":[ [20,10,20,100,30], ],   # Simon uses 30
   "Para_Set":[ "Li2023_ECdrag",],
   # solvent diffusion parameters:
   "EC initial concentration in electrolyte [mol.m-3]":[6250],
   "Typical EC concentration [mol.m-3]":[6250,],

   "Negative electrode porosity":[0.240507],
   "SEI resistivity [Ohm.m]":[2e5,],
   "Initial inner SEI thickness [m]":[1.2362e-08,],
   "Initial outer SEI thickness [m]":[1.2362e-08,],
   "Contact resistance [Ohm]":[6e-3],
   "Cation transference number":[
    "electrolyte_transference_number_EC_DMC_1_1_Landesfeind2019_Con"],
   "Electrolyte conductivity [S.m-1]":[
    "electrolyte_conductivity_EC_DMC_1_1_Landesfeind2019_Con"],
   "Electrolyte diffusivity [m2.s-1]":[
    "diff_constant"], # 3e-10 constant
   "Initial Neg SOC":[0.8841301667966484,],
   "Initial Pos SOC":[0.23552755074598045,],
   # SEI
   "Inner SEI lithium interstitial diffusivity [m2.s-1]":[8e-19,], 
}

Para_dict_DD_ONLY = {
     "Model option":[{
      "SEI":"interstitial-diffusion limited",            
      "SEI film resistance":"distributed",          
      "SEI porosity change":"true",   
      #"SEI on cracks": "true",
      "solvent diffusion": "double spatial consume w refill",
      "electrolyte conductivity": "sol full"  ,
     "contact resistance":"true",  
      },],
     ################################# Need to be different between single and double:
     "EC Lithium ion cross diffusivity [m2.s-1]":[
        "Cross_diffusivity_1p5E_12",], # 
     "EC diffusivity in electrolyte [m2.s-1]":[
        "EC_diffusivity_5E_10","EC_diffusivity_3E_10",
        "EC_diffusivity_1E_10","EC_diffusivity_5E_11"],
     "EC transference number":[
        "EC_transference_number",],
     "Measured dLJP_dcEC": ["dLJP_Two_Species_dco_Jung2023",],
     "Measured dLJP_dce": ["dLJP_Two_Species_dce_Jung2023",],
}

Para_dict_DD = {**Para_dict_Same,**Para_dict_DD_ONLY}
Para_DD = [];  Para_SD=[];
recursive_scan(Para_DD,Para_dict_DD, list(Para_dict_DD.keys()), {})
print(len(Para_DD),len(Para_SD))

4 0


In [15]:
book_name_xlsx = 'Test.xlsx';sheet_name_xlsx = 'Results';
Path_Exp_Crate = "D:/OneDrive - Imperial College London/SimDataSave/InputData/Ruihe_newLGM50_Crate/"
Path_pack = [BasicPath,Target,Path_Exp_Crate,book_name_xlsx,sheet_name_xlsx,];

In [16]:
Rate_Dis_All = np.arange(1,4.5+1.4,1.4) # np.arange(3.8,4.5+0.175,0.175)
print(Rate_Dis_All)

[1.  2.4 3.8 5.2]


In [17]:
# Three set of solution:
#Rate_Dis_All = [ 1,  2.5,  4 , 4.5 ]; # 5 stuck 4.5 works
#Rate_Dis_All = np.arange(1,4.5+0.175,0.7) # np.arange(1,4.5+0.175,0.175) # [ 1, 1.75,2,2.25, 2.5, 3, 3.25, 3.5, 3.75, 4 ,4.2,4.4, 4.5 ];
EC_DD_HDx_Crate = [];
for i,para_dd in enumerate(Para_DD):
    EC_DD_HDx_Crate.append(Scan_Crate_Paper(
        i, para_dd, Path_pack , "DD_HDx",
        Rate_Dis_All,True,))

Start Now! Scan 0.
Start Now! C rate: 1.0
Confirm: using interstitial-diffusion limited


In [ ]:
# cyc based 
def Get_Vol_Component(DD_HDx_Crate):
    DD_HDx_Crate['Cyc Mean eta_c_e'] = []
    DD_HDx_Crate['Cyc Mean eta_c_EC'] = []
    DD_HDx_Crate['Cyc Mean eta_others'] = []
    DD_HDx_Crate['Cyc Mean eta_tot'] = []
    DD_HDx_Crate['Cyc Last eta_c_e'] = []
    DD_HDx_Crate['Cyc Last eta_c_EC'] = []
    DD_HDx_Crate['Cyc Last eta_others'] = []
    DD_HDx_Crate['Cyc Last eta_tot'] = []
    DD_HDx_Crate['Cyc Mean eta_c_e %'] = []
    DD_HDx_Crate['Cyc Mean eta_c_EC %'] = []
    DD_HDx_Crate['Cyc Mean eta_others %'] = []
    for sol in DD_HDx_Crate['Sol_All']:
        step_CD = sol.cycles[0].steps[1]
        ocv = step_CD["X-averaged battery open circuit voltage [V]"].entries
        eta_c_e = -step_CD[
            "X-averaged battery concentration overpotential [V]"].entries
        eta_c_EC = -step_CD[
            "X-averaged EC concentration overpotential [V]"].entries
        V = step_CD["Terminal voltage [V]"].entries
        eta_others = ocv - eta_c_e - eta_c_EC - V
        eta_tot = ocv - V
        mean_eta_c_e = np.mean(eta_c_e)
        mean_eta_c_EC = np.mean(eta_c_EC)
        mean_eta_others = np.mean(eta_others)
        mean_eta_c_tot = np.mean(eta_tot)
        DD_HDx_Crate['Cyc Mean eta_c_e'].append(np.mean(eta_c_e))
        DD_HDx_Crate['Cyc Mean eta_c_EC'].append(np.mean(eta_c_EC))
        DD_HDx_Crate['Cyc Mean eta_others'].append(np.mean(eta_others))
        DD_HDx_Crate['Cyc Mean eta_tot'].append(np.mean(eta_tot))
        DD_HDx_Crate['Cyc Last eta_c_e'].append(eta_c_e[-1])
        DD_HDx_Crate['Cyc Last eta_c_EC'].append(eta_c_EC[-1])
        DD_HDx_Crate['Cyc Last eta_others'].append(eta_others[-1])
        DD_HDx_Crate['Cyc Last eta_tot'].append(eta_tot[-1])
        DD_HDx_Crate['Cyc Mean eta_c_e %'].append(
            mean_eta_c_e/mean_eta_c_tot*100)
        DD_HDx_Crate['Cyc Mean eta_c_EC %'].append(
            mean_eta_c_EC/mean_eta_c_tot*100)
        DD_HDx_Crate['Cyc Mean eta_others %'].append(
            mean_eta_others/mean_eta_c_tot*100)
    return DD_HDx_Crate
DD_HDx_Crate = Get_Vol_Component(DD_HDx_Crate)
DD_LDx_Crate = Get_Vol_Component(DD_LDx_Crate)
SD_Crate     = Get_Vol_Component(SD_Crate)

In [ ]:
# Fig. 3 - only overpotential, and three components:
def Plot_Voldecom_All(DD_HDx_Crate,Crate_index,SaveFig):
    #Crate_index = -1
    Colors_a = [
        [31/255, 119/255, 180/255, 0.5], # like blue
        [44/255, 160/255, 44/255,  0.7], # like green  
        [214/255, 39/255, 40/255,  0.7], # like red
        [255/255, 127/255, 14/255, 0.9],]# like orange 
    sol_dd_HDx = DD_HDx_Crate['Sol_All'][Crate_index]
    step_CD = sol_dd_HDx.cycles[0].steps[1]
    time = step_CD["Time [s]"].entries - step_CD["Time [s]"].entries[0] # 
    zeros = 0*np.ones_like(time)
    ocv = step_CD["X-averaged battery open circuit voltage [V]"].entries
    Total_overpotential = ocv - step_CD["Terminal voltage [V]"].entries;
    eta_c_e = -step_CD["X-averaged battery concentration overpotential [V]"].entries
    eta_c_EC = -step_CD["X-averaged EC concentration overpotential [V]"].entries
    V = step_CD["Terminal voltage [V]"].entries
    eta_others = ocv - eta_c_e - eta_c_EC - V
    eta_tot = ocv - V

    fs = 12; font = {'family' : 'Times New Roman','size'   : fs}
    mpl.rc('font', **font)
    fig, ax = plt.subplots(figsize=(9/2.54,6/2.54),tight_layout=True)
    ax.plot(time, ocv, "k--", ) # label="Open-circuit voltage"
    ax.fill_between(time, ocv-eta_c_e, ocv, color=Colors_a[1], label=r"Li$^+$")
    ax.fill_between(time, ocv-eta_c_e-eta_c_EC, ocv-eta_c_e, color=Colors_a[3],  label="EC")
    ax.fill_between(time, V, ocv-eta_c_e-eta_c_EC,  color=Colors_a[0],  label="Others")

    ax.plot(time, V, "k-", ) # label="Terminal voltage"

    leg = ax.legend(loc="lower left", bbox_to_anchor=(-0.01, -0.07),frameon=False)
    ax.set_xlim([time[0], time[-1]])
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("Voltage [V]")
    #ax.set_ylabel(r"$\vec \nabla \cdot \vec N_\mathrm{diff}$") 
    ax.set_ylim([2.3,4.3])
    plt.show()
    if SaveFig == True:
        plt.savefig(BasicPath + Target
            +f"{Rate_Dis_All[Crate_index]}C discharge voltage decomposition.png", 
            dpi=1000)
        plt.savefig(BasicPath + Target + 
            f'{Rate_Dis_All[Crate_index]}C discharge voltage decomposition.svg') 
#Plot_Voldecom_All(SD_Crate,-1,False)
#Plot_Voldecom_All(DD_LDx_Crate,-1,False)
Plot_Voldecom_All(DD_HDx_Crate,-1,  False)

In [ ]:
# plot EC concentraton change 
def Plot_Loc_Var_sol_10( sol,x_loc_all, key_all,titles,YlimAll, cycle, step,colormap,fs): # for initial solution object
    font = {'family' : 'Times New Roman','size'   : fs}
    mpl.rc('font', **font)
    #[key_Li,key_EC] = key_all
    Num_subplot = len(key_all[0]); # must have 2+ keys
    fig, axs = plt.subplots(2,Num_subplot, figsize=(30/2.53,12/2.54),tight_layout=True)

    for m in range(0,len(key_all)):
        keys = key_all[m]
        Ylim = YlimAll[m]
        for n in range(0,Num_subplot):
            x_loc=x_loc_all[n]; 
            key=keys[n]; 
            temp_L_Num= len(sol.cycles[cycle].steps[step][key].entries[0,:] )
            #print(f"Total line number is: {temp_L_Num}")
            if temp_L_Num > 26:
                xx = np.arange(0,temp_L_Num,int(np.rint(temp_L_Num/50)));
            else:
                xx = np.arange(0,temp_L_Num,1);
            xx = xx.tolist()
            if not xx[-1]==temp_L_Num-1:
                xx.append(temp_L_Num-1)
            cmap_i = mpl.cm.get_cmap(colormap, len(xx)) 
            for index_j,j in zip(  range(0,len(xx)), xx):
                len_1 = len(sol.cycles[cycle].steps[step][x_loc].entries[:,0])
                len_2 = len(sol.cycles[cycle].steps[step][key].entries[:,j])
                if len_1>len_2:
                    axs[m,n].plot(
                        sol.cycles[cycle].steps[step][x_loc].entries[0:len_2,0]*1e6, 
                        sol.cycles[cycle].steps[step][key].entries[:,j], '-',
                        color=cmap_i(index_j),)
                elif len_1<len_2:
                    axs[m,n].plot(
                        sol.cycles[cycle].steps[step][x_loc].entries[:,0]*1e6, 
                        sol.cycles[cycle].steps[step][key].entries[0:len_1,j], '-',
                        color=cmap_i(index_j),)
                else:    
                    axs[m,n].plot(
                        sol.cycles[cycle].steps[step][x_loc].entries[:,0]*1e6, 
                        sol.cycles[cycle].steps[step][key].entries[:,j], '-',
                        color=cmap_i(index_j),)
            Y_0p = np.arange(-1e8,1e8,5e7)
            axs[m,n].plot( 85.2 * np.ones_like(Y_0p),Y_0p,"k-",linewidth=0.5,)
            axs[m,n].plot((85.2+12 )* np.ones_like(Y_0p),Y_0p, "k-",linewidth=0.5,)
            axs[m,n].set_ylim(Ylim[n])
            axs[1,n].set_xlabel(r"x [$\mu$m]",   fontdict={'family':'Times New Roman','size':fs})
            """ axs[m,n].ticklabel_format( 
            axis='y', style='sci',
            scilimits=[-1e-6,1e-6], useOffset=None, 
            useLocale=None, useMathText=True) """
            axs[m,n].tick_params(
                labelcolor='k', labelsize=fs, width=1) ; 
            labels = axs[m,n].get_xticklabels() + axs[m,n].get_yticklabels(); 
            [label.set_fontname('Times New Roman') for label in labels]
            axs[0,n].set_title(titles[n] ,   fontdict={'family':'Times New Roman','size':fs+1})
    axs[0,0].set_ylabel(r"c [mol/m$^{3}$]")
    axs[1,0].set_ylabel(r"c [mol/m$^{3}$]")
    axs[0,1].set_ylabel(r"$\partial(\epsilon c)$/$\partial$(t) [mol/m$^{3}$/s]")
    axs[1,1].set_ylabel(r"$\partial(\epsilon c)$/$\partial$(t) [mol/m$^{3}$/s]")
    
    return  fig, axs 

In [ ]:
# Plot concentration flux and source - 4.5C high Dx
x_loc_all = ["x [m]"]*5
key_Li = [
    "Electrolyte concentration [mol.m-3]",
    "Li+ source term [mol.m-3.s-1]",
    "y_e",
    "Minus div Li+ flux by migration [mol.m-3.s-1]",
    "Minus div Li+ flux by solvent [mol.m-3.s-1]",
]
key_EC = [
    "EC concentration [mol.m-3]",
    "EC source term (SEI) [mol.m-3.s-1]",
    "y_EC",
    "Minus div EC flux by migration [mol.m-3.s-1]",
    "Minus div EC flux by Li+ [mol.m-3.s-1]",
]
titles = [
    "Con.",#r"Li$^+$",
    "Source",
    "y",
    #r"$\vec \nabla \cdot \vec N_\mathrm{diff}$",
    r"$\vec \nabla \cdot \vec N_\mathrm{migr}$",
    r"$\vec \nabla \cdot \vec N_\mathrm{cross}$"
]
key_all = [key_Li,key_EC];
YlimAll = [
    [[-5e2,4e3],[-6e3,8e3],[0.15,0.155],[-4e3,3e3],[-3e2,4e2]],
    [[3.8e3,8e3],[-3e1,1e-4],[0.3,0.8],[-2.5e4,2.5e4],[-5e3,5e3]]
]
for i in range(len(Rate_Dis_All)):
    Crate=i
    sol = SD_Crate['Sol_All'][Crate]
    #step_CD = sol_dd_HDx.cycles[0].steps[1]
    fig, axs= Plot_Loc_Var_sol_10( sol,x_loc_all, key_all,titles,YlimAll, 0, -1,"cool",12)
    str = f"Crate={Rate_Dis_All[Crate]}Single _"
    """ plt.savefig(
    BasicPath + Target +    str +
    f"- c(Li+) and flux.png", dpi=1000)
    plt.savefig(
    BasicPath + Target +    str +
    f"- c(Li+) and flux.svg")  """

In [ ]:
# Fig. 4 Concentration, LJP and overpotential 
def FindClose(time_DD,t_select): # Input must be a list
    for i in range(0,len(time_DD)):
        if abs(time_DD[i] - t_select)<1e-3:
            break
    return i
fs = 12
font = {'family' : 'Times New Roman','size'   : fs}
mpl.rc('font', **font)
x_loc = ["x [m]","x [m]","x [m]","x [m]","x [m]","x [m]"]
Str=[ 'DFN',   'Double trans' ]; # Str_model =[ 'Model_DFN',   'Model_DD' ]
Colors = ['#1f77b4','#ff7f0e',  '#d62728', ];
Labels = ["Single",r"Double-Low $D_\times$",r"Double-High $D_\times$"]
LS = ['-','--','-',];MS=['none','none','h']; ms = 8
Str=[ 'DFN',  'Double' ]
def Plot_Eta_c(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,Ylim,Crate_index,SaveFig):
    fig, axs = plt.subplots(2,3, figsize=(22/2.53,12/2.54),tight_layout=True) #,
    step_sd = SD_Crate['Sol_All'][Crate_index].cycles[0].steps[1]
    step_DD_LDx = DD_LDx_Crate['Sol_All'][Crate_index].cycles[0].steps[1]
    step_DD_HDx = DD_HDx_Crate['Sol_All'][Crate_index].cycles[0].steps[1]
    Steps = [step_sd,step_DD_LDx,step_DD_HDx] 
    Labels = [r"Single",r"Double-Low $\mathit{D}_\times$",r"Double-High $\mathit{D}_\times$"]
    for k,step in enumerate(Steps):
        axs[0,2].plot(
            step['Time [s]'].entries-step['Time [s]'].entries[0], 
            -step["X-averaged battery concentration overpotential [V]"].entries*1e3,
            color=Colors[k],linestyle=LS[k],label=Labels[k]) 
        axs[1,2].plot(
            step['Time [s]'].entries-step['Time [s]'].entries[0], 
            -step["X-averaged battery EC concentration overpotential [V]"].entries*1e3,
            color=Colors[k],linestyle=LS[k],label=Labels[k]) 
    axs[0,2].set_ylabel("$\eta_{\mathrm{Li}^+}$ [mV]",fontsize=fs)
    #"$\partial\Delta$U/$\partial$c$_\mathrm{EC}$"
    axs[1,2].set_ylabel("$\eta_{\mathrm{EC}}$ [mV]",fontsize=fs)
    axs[1,2].set_xlabel("Time [s]",fontsize=fs)
    axs[0,2].set_xlabel("Time [s]",fontsize=fs)
    #fig.suptitle(f"EC and Li+ overpotential - {Rate_Dis_All[Crate_index]}C Discharge", fontsize=fs+1)
    var_elely = [
        "Electrolyte concentration [mol.m-3]",
        "dLJP_dce",
        "EC concentration [mol.m-3]",
        "dLJP_dcEC",] 
    Title_elely = [
        "c(Li+) [M]",
        "dLJP_dce",
        "c(EC) [M]",
        "dLJP_dcEC",]

    time_Single_High = (step_sd['Time [s]'].entries-step_sd['Time [s]'].entries[0]).tolist()
    time_Double_High = (step_DD_HDx['Time [s]'].entries-step_DD_HDx['Time [s]'].entries[0]).tolist()
    time_Double_Low  = (step_DD_LDx['Time [s]'].entries-step_DD_LDx['Time [s]'].entries[0]).tolist()

    t_select = min(time_Single_High[-2],time_Double_High[-2],time_Double_Low[-2])
    i_Single_High = FindClose(time_Single_High,t_select)
    i_Double_High = FindClose(time_Double_High,t_select)

    i_Double_Low = FindClose(time_Double_Low,t_select)

    print(f'Time selected is for {Rate_Dis_All[Crate_index]}C is {time_Single_High[i_Single_High]} and {time_Double_High[i_Double_High]},{time_Double_Low[i_Double_Low]} s')
    Y_0p = np.arange(-1,100,10)
    Index_MN = [ [0,0],[0,1],[1,0],[1,1]     ]
    for i in range(4):
        m,n = Index_MN[i]
        x_Single_High =   step_sd[x_loc[i]].entries[:,0]
        y_Single_High =   step_sd[var_elely[i]].entries[:,i_Single_High]

        x_Double_High =   step_DD_HDx[x_loc[i]].entries[:,0]
        y_Double_High =   step_DD_HDx[var_elely[i]].entries[:,i_Double_High]
        x_Double_Low  =   step_DD_LDx[x_loc[i]].entries[:,0]
        y_Double_Low  =   step_DD_LDx[var_elely[i]].entries[:,i_Double_Low]
        if var_elely[i].__contains__("[mol.m-3]"): 
            axs[m,n].plot(x_Single_High*1e6, y_Single_High/1e3, color=Colors[0],linestyle=LS[0],label=r"Single") 
            axs[m,n].plot(x_Double_Low*1e6, y_Double_Low/1e3, color=Colors[1],linestyle=LS[1],label=r"Double-Low D$_\times$") 
            axs[m,n].plot(x_Double_High*1e6, y_Double_High/1e3, color=Colors[2],linestyle=LS[2],label=r"Double-High D$_\times$") 
            axs[m,n].plot( 85.2 * np.ones_like(Y_0p),Y_0p,"k-",linewidth=0.5,)
            axs[m,n].plot((85.2+12 )* np.ones_like(Y_0p),Y_0p, "k-",linewidth=0.5,)
        else:
            if var_elely[i] == "dLJP_dcEC": 
                pass
            else:
                axs[m,n].plot(x_Single_High*1e6, y_Single_High, color=Colors[0],linestyle=LS[0],label=r"Single") 
            axs[m,n].plot(x_Double_Low*1e6, y_Double_Low, color=Colors[1],linestyle=LS[1],label=r"Double-Low $D_\times$") 
            axs[m,n].plot(x_Double_High*1e6, y_Double_High, color=Colors[2],linestyle=LS[2],label=r"Double-High $D_\times$") 
            axs[m,n].plot( 85.2 * np.ones_like(Y_0p),Y_0p,"k-",linewidth=0.5,)
            axs[m,n].plot((85.2+12 )* np.ones_like(Y_0p),Y_0p, "k-",linewidth=0.5,)

        axs[m,n].set_xlabel(r"x [$\mu$m]",   fontdict={'family':'Times New Roman','size':fs})
        #axs[m,n].set_title(Title_elely[i]) 
        axs[m,n].ticklabel_format( 
            axis='x', style='sci',
            scilimits=[-1e2,1e2], useOffset=None, 
            useLocale=None, useMathText=True)
        axs[m,n].ticklabel_format( 
            axis='y', style='sci',
            scilimits=[-1e-2,1e-2], useOffset=None, 
            useLocale=None, useMathText=True)
    axs[0,0].set_ylabel("c(Li$^+$) [M]");
    axs[1,0].set_ylabel("c(EC) [M]")
    axs[0,1].set_ylabel("$\partial\Delta$U/$\partial$c$_\mathrm{e}$");  
    axs[0,1].set_yscale("log")
    axs[1,1].set_ylabel("$\partial\Delta$U/$\partial$c$_\mathrm{EC}$")

    axs[0,0].set_ylim(Ylim[0])
    axs[0,1].set_ylim(Ylim[1])
    axs[1,0].set_ylim(Ylim[2])
    axs[1,1].set_ylim(Ylim[3])
    axs[0,2].set_ylim(Ylim[4])
    axs[1,2].set_ylim(Ylim[5])
    fig.subplots_adjust(wspace=0, hspace=0)
    """ axs[1,2].legend(prop={'family':'Times New Roman','size':fs-0.5},
    loc='upper right',ncol=1,  bbox_to_anchor=(0.92, 0.7),
    frameon=False,markerscale=0.5,borderpad=0.0,
    handlelength=1.4,columnspacing=1) """
    if SaveFig == True:
        plt.savefig(BasicPath + Target
                +f"Fig_4_{Rate_Dis_All[Crate_index]}C Overpotential and concentration.png", dpi=1000)
        plt.savefig(BasicPath + Target + 
                f"Fig_4_{Rate_Dis_All[Crate_index]}C Overpotential and concentration.svg") 

fs = 12;font = {'family' : 'Times New Roman','size': fs}
mpl.rc('font', **font)
LS = ['-','--','-',]; 
# for Crate_index=-1 - 4.5C
Ylim = [[-0.1,4.5],[0.5e-4,1.0e2],[3.6,8],[-5e-5,-0.3e-5],[-10,360],[-0.5,17]]
# for Crate_index=-4 - 4C
#Ylim = [[-0.1,4.5],[0.5e-4,2.0e-2],[3.6,8],[-2.e-5,-0.3e-5]]
# for Crate_index=5 -  3C
# Ylim = [[-0.1,4.5],[0.5e-4,3.0e-4],[3.6,8],[-1.2e-5,-0.3e-5]]
for i in range(len(Rate_Dis_All)):
    Plot_Eta_c(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,Ylim,Crate_index=i,SaveFig=False)
#Plot_Eta_c(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,Ylim,Crate_index=0,SaveFig=False)

In [ ]:
step = DD_HDx_Crate['Sol_All'][-1].cycles[0].steps[1]
step["dLJP_dcEC"].entries[:,-1]

In [ ]:
# for sol_high_Dx but different C rate:

fs = 12
font = {'family' : 'Times New Roman','size'   : fs}
mpl.rc('font', **font)
x_loc = ["x [m]","x [m]","x [m]","x [m]","x [m]","x [m]"]
Str=[ 'DFN',   'Double trans' ]; # Str_model =[ 'Model_DFN',   'Model_DD' ]
Colors = ['#1f77b4','#ff7f0e',  '#d62728', ];
Labels = ["Single",r"Double-Low $D_\times$",r"Double-High $D_\times$"]
LS = ['-','--','-',];MS=['none','none','h']; ms = 8
Str=[ 'DFN',  'Double' ]

def Plot_Eta_OneSol(Sol_List,Ylim,str,SaveFig):

    def Plot_One_Crate(Sol_List,Crate_index,cm):
        step = Sol_List['Sol_All'][Crate_index].cycles[0].steps[1]

        axs[0,2].plot(
            step['Time [s]'].entries-step['Time [s]'].entries[0], 
            -step["X-averaged battery concentration overpotential [V]"].entries*1e3,
            color=cm(Crate_index)) 
        axs[1,2].plot(
            step['Time [s]'].entries-step['Time [s]'].entries[0], 
            -step["X-averaged battery EC concentration overpotential [V]"].entries*1e3,
            color=cm(Crate_index)) 
    
        #fig.suptitle(f"EC and Li+ overpotential - {Rate_Dis_All[Crate_index]}C Discharge", fontsize=fs+1)
        var_elely = [
            "Electrolyte concentration [mol.m-3]",
            "dLJP_dce",
            "EC concentration [mol.m-3]",
            "dLJP_dcEC",] 
        Title_elely = [
            "c(Li+) [M]",
            "dLJP_dce",
            "c(EC) [M]",
            "dLJP_dcEC",]

        Y_0p = np.arange(-1,100,10)
        Index_MN = [ [0,0],[0,1],[1,0],[1,1]     ]
        for i in range(4):
            m,n = Index_MN[i]
            x_Single_High =   step[x_loc[i]].entries[:,0]
            y_Single_High =   step[var_elely[i]].entries[:,-1]
            if var_elely[i].__contains__("[mol.m-3]"): 
                axs[m,n].plot(x_Single_High*1e6, y_Single_High/1e3, color=cm(Crate_index)) 
                axs[m,n].plot( 85.2 * np.ones_like(Y_0p),Y_0p,"k-",linewidth=0.5,)
                axs[m,n].plot((85.2+12 )* np.ones_like(Y_0p),Y_0p, "k-",linewidth=0.5,)
            else:
                axs[m,n].plot(x_Single_High*1e6, y_Single_High, color=cm(Crate_index)) 
                axs[m,n].plot( 85.2 * np.ones_like(Y_0p),Y_0p,"k-",linewidth=0.5,)
                axs[m,n].plot((85.2+12 )* np.ones_like(Y_0p),Y_0p, "k-",linewidth=0.5,)

            axs[m,n].set_xlabel(r"x [$\mu$m]",   fontdict={'family':'Times New Roman','size':fs})
            #axs[m,n].set_title(Title_elely[i]) 
            axs[m,n].ticklabel_format( 
                axis='x', style='sci',
                scilimits=[-1e2,1e2], useOffset=None, 
                useLocale=None, useMathText=True)
            axs[m,n].ticklabel_format( 
                axis='y', style='sci',
                scilimits=[-1e-2,1e-2], useOffset=None, 
                useLocale=None, useMathText=True)
            
    fig, axs = plt.subplots(2,3, figsize=(22/2.53,12/2.54),tight_layout=True) #,
    for i in range(len(Rate_Dis_All)):
        Crate_index = i; 
        cm = mpl.cm.get_cmap("copper", len(Rate_Dis_All))  
        Plot_One_Crate(Sol_List,Crate_index,cm)
    axs[0,2].set_ylabel("$\eta_{\mathrm{Li}^+}$ [mV]",fontsize=fs)
    #"$\partial\Delta$U/$\partial$c$_\mathrm{EC}$"
    axs[1,2].set_ylabel("$\eta_{\mathrm{EC}}$ [mV]",fontsize=fs)
    axs[1,2].set_xlabel("Time [s]",fontsize=fs)
    axs[0,2].set_xlabel("Time [s]",fontsize=fs)

    axs[0,0].set_ylabel("c(Li$^+$) [M]");
    axs[1,0].set_ylabel("c(EC) [M]")
    axs[0,1].set_ylabel("$\partial\Delta$U/$\partial$c$_\mathrm{e}$");  
    axs[1,1].set_ylabel("$\partial\Delta$U/$\partial$c$_\mathrm{EC}$")
    axs[0,1].set_yscale("log")
    #axs[1,1].set_yscale("log")

    axs[0,0].set_ylim(Ylim[0])
    axs[0,1].set_ylim(Ylim[1])
    axs[1,0].set_ylim(Ylim[2])
    axs[1,1].set_ylim(Ylim[3])
    axs[0,2].set_ylim(Ylim[4])
    axs[1,2].set_ylim(Ylim[5])
    fig.subplots_adjust(wspace=0, hspace=0)
    """ axs[1,2].legend(prop={'family':'Times New Roman','size':fs-0.5},
    loc='upper right',ncol=1,  bbox_to_anchor=(0.92, 0.7),
    frameon=False,markerscale=0.5,borderpad=0.0,
    handlelength=1.4,columnspacing=1) """
    if SaveFig == True:
        plt.savefig(BasicPath + Target
                +f"Case_{str} Overpotential and concentration.png", dpi=1000)
        plt.savefig(BasicPath + Target + 
                f"Case_{str} Overpotential and concentration.svg") 

fs = 12;font = {'family' : 'Times New Roman','size': fs}
mpl.rc('font', **font)
LS = ['-','--','-',]; 
# for Crate_index=-1 - 4.5C
Ylim = [[-0.1,4.5],[0.5e-4,5.0e1],[3.6,8],[-5e-5,-0.0e-5],[0,360],[-0.5,16]]
# for Crate_index=-4 - 4C
#Ylim = [[-0.1,4.5],[0.5e-4,2.0e-2],[3.6,8],[-2.e-5,-0.3e-5]]
# for Crate_index=5 -  3C
# Ylim = [[-0.1,4.5],[0.5e-4,3.0e-4],[3.6,8],[-1.2e-5,-0.3e-5]]
# SD_Crate DD_LDx_Crate,DD_HDx_Crate
Plot_Eta_OneSol(DD_HDx_Crate,Ylim,"DD_HDx_Crate",SaveFig=True)
#Plot_Eta_c(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,Ylim,Crate_index=0,SaveFig=False)

In [ ]:
# Fig. 5 - Plot summary and overpotential 
def Plot_Summary_VolCom_Last(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,SaveFig):
    fig, axs = plt.subplots(2,2 , figsize=(18/2.54,13/2.54),layout='constrained') #
    Cases = [SD_Crate,DD_LDx_Crate,DD_HDx_Crate]
    for k,case in enumerate(Cases):
            y_temp = case['Cap_Dis_All']; 
            y_plt = (1 - y_temp / y_temp[0])*100
            axs[0,0].plot(
                Rate_Dis_All,y_plt,color=Colors[k],
                linestyle=LS[k],label=Labels[k]) 
            axs[0,1].plot(
                Rate_Dis_All,np.array(case['Cyc Last eta_tot'])*1e3,
                color=Colors[k],linestyle=LS[k],label=Labels[k]) 
            axs[1,0].plot(
                Rate_Dis_All,np.array(case['Cyc Last eta_c_e'])*1e3,
                color=Colors[k],linestyle=LS[k],label=Labels[k]) 
            axs[1,1].plot(
                Rate_Dis_All,np.array(case['Cyc Last eta_c_EC'])*1e3,
                color=Colors[k],linestyle=LS[k],label=Labels[k]) 
    axs[0,0].set_ylabel("Capacity loss %")
    axs[0,1].set_ylabel("$\eta_{\mathrm{tot}}$ [mV]",fontsize=fs)
    axs[1,0].set_ylabel("$\eta_{\mathrm{Li^+}}$ [mV]",fontsize=fs)
    axs[1,1].set_ylabel("$\eta_{\mathrm{EC}}$ [mV]",fontsize=fs)
    for k in range(2):
         axs[1,k].set_xlabel("Discharge rate")
    axs[0,0].legend(prop={'family':'Times New Roman','size':fs-1},
        loc='best',ncol=1,  #bbox_to_anchor=(1, -0.00),
        frameon=False,markerscale=0.5,borderpad=0.0,
        handlelength=1.0,columnspacing=1)
    if SaveFig == True:
        plt.savefig(BasicPath +  Target+ 
            f"Fig_5_Summary Elely Overpotential last.png", dpi=1000)
        plt.savefig(BasicPath +  Target+ 
            f"Fig_5_Elely Overpotential last.svg")  
fs = 16
font = {'family' : 'Times New Roman','size'   : fs}
mpl.rc('font', **font)
Str=[ 'DFN',   'Double trans' ]; # Str_model =[ 'Model_DFN',   'Model_DD' ]
Colors = ['#1f77b4','#ff7f0e',  '#d62728', ];
Labels = ["Single",r"Double-Low $D_\times$",r"Double-High $D_\times$"]
LS = ['-','--','-',];MS=['none','none','h']; ms = 8
Plot_Summary_VolCom_Last(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,SaveFig=True)

In [ ]:
953.1-878.2

In [ ]:
Cases = [SD_Crate,DD_LDx_Crate,DD_HDx_Crate]
y_plt_all = [];
for i in range(3):
    y_temp = Cases[i]['Cap_Dis_All']; 
    y_plt = (1 - y_temp / y_temp[0])*100
    y_plt_all.append(y_plt)
for i in range(len(Rate_Dis_All)):
    print(
    round(Rate_Dis_All[i],3),round(y_plt_all[0][i],1),
    round(y_plt_all[1][i],1),round(y_plt_all[2][i],1),)
    # case['Cyc Last eta_tot']
    #print(
    #    round(Rate_Dis_All[i],3),
    #    round(Cases[0]['Cyc Last eta_tot'][i],4),
    #    round(Cases[1]['Cyc Last eta_tot'][i],4),
    #    round(Cases[2]['Cyc Last eta_tot'][i],4),)

In [ ]:
# Plot EC:DMC wt%
def Plot_EC_DMC_Crate(DD_HDx_Crate,SaveFig):
    cm = mpl.cm.get_cmap("cool", len(DD_HDx_Crate['Sol_All'])) 

    Y_0p = np.arange(-1,10,0.1)
    fig, axs = plt.subplots( figsize=(8/2.54,4.5/2.54), ) #tight_layout=True
    for i,sol_Double_High in enumerate(DD_HDx_Crate['Sol_All']):
        step_i  =sol_Double_High.cycles[0].steps[1]
        t_i = (step_i['Time [s]'].entries-step_i['Time [s]'].entries[0]).tolist()
        x_Double_High =   step_i["x [m]"].entries[:,0]
        y_Double_High =   step_i["EC:EMC wt%"].entries[:,-1]
        axs.plot(x_Double_High*1e6, y_Double_High, 
                 color=cm(i),linestyle=LS[0],
                 label=r"Double-High D$_\times$") 

        axs.set_xlabel(r"x [$\mu$m]",fontdict={'family':'Times New Roman','size':fs})
        #axs.set_title(Title_keys[i]) 
        axs.ticklabel_format( 
            axis='x', style='sci',
            scilimits=[-1e3,1e3], useOffset=None, 
            useLocale=None, useMathText=None)
    axs.set_ylabel("EC:EMC wt%")
    #axs.legend(prop={'family':'Times New Roman','size':fs-3},loc='best',frameon=False)  
    #axs[0].set_ylim([-0.1,4.1])
    #axs[1].set_ylim([1.7,4.8])
    #axs[2].set_ylim([5.5,7.6])
    # fig.suptitle(f"3C Discharge, t={round(t_select)} s", fontsize=fs+1)
    if SaveFig == True:
        plt.savefig(BasicPath +  Target+ f"c.png", dpi=1000)
        plt.savefig(BasicPath +  Target+ f"EC:EMC wt%.svg")  

Plot_EC_DMC_Crate(DD_HDx_Crate,SaveFig=False)


In [ ]:
# plot the effect of charge / discharge rate and Dx
fs = 12
font = {'family' : 'Times New Roman','size'   : fs}
mpl.rc('font', **font)
Str=[ 'DFN',   'Double trans' ]; # Str_model =[ 'Model_DFN',   'Model_DD' ]

Colors = ['#1f77b4','#ff7f0e',  '#d62728', ];
Labels = ["Single",r"Double-Low $D_\times$",r"Double-High $D_\times$"]
LS = ['-','--','-',];MS=['none','none','h']; ms = 8

def Plot_Summary(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,SaveFig):
        fig, axs = plt.subplots( figsize=(9/2.54,6/2.54), layout='constrained')
        Cases = [SD_Crate,DD_LDx_Crate,DD_HDx_Crate]
        for k,case in enumerate(Cases):
                y_temp = case['Cap_Dis_All']; 
                y_plt = (1 - y_temp / y_temp[0])*100
                axs.plot(
                        Rate_Dis_All,y_plt,color=Colors[k],
                        linestyle=LS[k],label=Labels[k]) 
        axs.set_ylabel("Capacity loss %")
        axs.set_xlabel("Discharge rate")
        axs.set_xlabel("Discharge rate")
        #axs.set_ylim(3.9,5.1);   # axs[1].set_ylim(3.9,5.1)

        #axs[0].set_title(r"$D_\times=1.5\bullet10^{-12}  m^2/s$", fontdict={'family':'Times New Roman','size':fs})
        #axs[1].set_title(r"$D_\times=1.5\bullet10^{-10}  m^2/s$", fontdict={'family':'Times New Roman','size':fs})
        axs.legend(prop={'family':'Times New Roman','size':fs},loc='best',frameon=False,)
        #axs.legend(prop={'family':'Times New Roman','size':fs},loc='best',frameon=False,)
        if SaveFig == True:
                plt.savefig(
                        BasicPath + Target+"Fig_3-Summary Crate.png", dpi=1000)
                plt.savefig(BasicPath + Target + 
                        f'Fig_3-Summary Crate.svg') 
Plot_Summary(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,SaveFig=True)

In [ ]:
# plot voltage stuff:
fs = 17
font = {'family' : 'Times New Roman','size'   : fs}
mpl.rc('font', **font)
LS = ['-','--','-',];
def Plot_Voldecom_All(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,Crate_index,SaveFig):
    # Crate_index = -1
    sol_sd = SD_Crate['Sol_All'][Crate_index]
    sol_dd_HDx = DD_HDx_Crate['Sol_All'][Crate_index]
    sol_dd_LDx = DD_LDx_Crate['Sol_All'][Crate_index]
    Sol_All = [sol_sd,sol_dd_LDx,sol_dd_HDx,];
    Str = ["Single",r"Double low D$_\times$",r"Double high D$_\times$"]
    fig, axs = plt.subplots(2,3, figsize=(12,7),tight_layout=True)
    Keys_t_var = [
        "Battery voltage [V]",
        "X-averaged battery reaction overpotential [V]",
        "X-averaged battery concentration overpotential [V]",
        "X-averaged EC concentration overpotential [V]", # Mark Ruihe add
        "X-averaged battery electrolyte ohmic losses [V]",
        "X-averaged battery solid phase ohmic losses [V]",
    ]
    Title_keys = [
        "Battery voltage",
        "Reaction overpotential",
        "c(Li+) overpotential",
        "c(EC) overpotential",
        "Elely ohmic losses",
        "Solid ohmic losses"
    ]
    for k,sol in enumerate(Sol_All):
        time_1 = sol.cycles[0].steps[1]['Time [s]'].entries-sol.cycles[0].steps[1]['Time [s]'].entries[0]
        for i in range(0,6): # 0 1 2 3 4 5 --> [0,0] [0,1], [0,2], [1,0], [1,1], [1,2]   
            var =  sol.cycles[0].steps[1][Keys_t_var[i]].entries
            if i < 3:
                m=0; n =i
            else:
                m=1; n=i-3
            axs[m,n].plot(time_1, var,color=Colors[k],linestyle=LS[k],label=Str[k]) # label="1st cycle"
            axs[m,n].set_title(Title_keys[i],fontdict={'family':'Times New Roman','size':fs-1})
            
            axs[1,n].set_xlabel('Time [s]',   fontdict={'family':'Times New Roman','size':fs})
            fig.suptitle(f"{Rate_Dis_All[Crate_index]}C Discharge", fontsize=fs+1)
            
            labels = axs[m,n].get_xticklabels() + axs[m,n].get_yticklabels(); 
            axs[0,0].legend(prop={'family':'Times New Roman','size':fs-3},loc='best',frameon=False)  
        axs[0,0].set_ylim(2.4,4.2)
    axs[0,0].set_ylabel("Potential [V]",   fontdict={'family':'Times New Roman','size':fs})
    axs[1,0].set_ylabel("Potential [V]",   fontdict={'family':'Times New Roman','size':fs})
    if SaveFig == True:
        plt.savefig(BasicPath + Target
                +f"{Rate_Dis_All[Crate_index]}C Vol decom all.png", dpi=1000)
        plt.savefig(BasicPath + Target + 
                f"{Rate_Dis_All[Crate_index]}C Vol decom all.svg") 
Plot_Voldecom_All(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,2,SaveFig=False)
Plot_Voldecom_All(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,-1,SaveFig=False)

In [ ]:
Colors = ['#1f77b4','#ff7f0e',  '#d62728', ];
LS = ['-','--','-',];MS=['none','none','h']; ms = 8
def Plot_Voldecom_mean_Eley(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,SaveFig):
    fig, axs = plt.subplots(1,3 , figsize=(18/2.54,5.5/2.54),layout='constrained') #
    axs[0].plot(Rate_Dis_All,SD_Crate['Cyc Mean eta_tot'],
                color=Colors[0],linestyle=LS[0],label=r"Single") # marker = MS[1],markerfacecolor='w',markersize = ms,
    axs[1].plot(Rate_Dis_All,SD_Crate['Cyc Mean eta_c_EC'],
                color=Colors[0],linestyle=LS[0],label=r"Single")
    axs[2].plot(Rate_Dis_All,SD_Crate['Cyc Mean eta_c_e'],
                color=Colors[0],linestyle=LS[0],label=r"Single")

    axs[0].plot(Rate_Dis_All,DD_LDx_Crate['Cyc Mean eta_tot'],
                color=Colors[1],linestyle=LS[1], label=r"Double low D$_\times$")
    axs[1].plot(Rate_Dis_All,DD_LDx_Crate['Cyc Mean eta_c_EC'],
        color=Colors[1],linestyle=LS[1],label=r"Double low D$_\times$")
    axs[2].plot(Rate_Dis_All,DD_LDx_Crate['Cyc Mean eta_c_e'],
        color=Colors[1],linestyle=LS[1],label=r"Double low D$_\times$")

    axs[0].plot(Rate_Dis_All,DD_HDx_Crate['Cyc Mean eta_tot'],
                color=Colors[2],linestyle=LS[2], label=r"Double high D$_\times$")
    axs[1].plot(Rate_Dis_All,DD_HDx_Crate['Cyc Mean eta_c_EC'],
        color=Colors[2],linestyle=LS[2],label=r"Double high D$_\times$")
    axs[2].plot(Rate_Dis_All,DD_HDx_Crate['Cyc Mean eta_c_e'],
        color=Colors[2],linestyle=LS[2],label=r"Double high D$_\times$")
    axs[0].set_ylabel("Overpotential [V]")
    axs[0].set_xlabel("Discharge rate")
    axs[1].set_xlabel("Discharge rate")
    axs[2].set_xlabel("Discharge rate")
    axs[0].set_title("Total") 
    axs[1].set_title("EC") 
    axs[2].set_title(r"Li$^+$") 
    if SaveFig == True:
        plt.savefig(BasicPath +  Target+ 
            f"Elely Overpotential mean.png", dpi=1000)
        plt.savefig(BasicPath +  Target+ 
            f"Elely Overpotential mean.svg")  
def Plot_Voldecom_last_Eley(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,SaveFig):
    fig, axs = plt.subplots(1,3 , figsize=(18/2.54,5.5/2.54),layout='constrained') #
    
    LS = ['-','--','-',];MS=['none','none','h']; ms = 8

    axs[0].plot(Rate_Dis_All,np.array(SD_Crate['Cyc Last eta_tot'])*1e3,
                color=Colors[0],linestyle=LS[0],label=r"Single") # marker = MS[1],markerfacecolor='w',markersize = ms,
    axs[1].plot(Rate_Dis_All,np.array(SD_Crate['Cyc Last eta_c_EC'])*1e3,
                color=Colors[0],linestyle=LS[0],label=r"Single")
    axs[2].plot(Rate_Dis_All,np.array(SD_Crate['Cyc Last eta_c_e'])*1e3,
                color=Colors[0],linestyle=LS[0],label=r"Single")

    axs[0].plot(Rate_Dis_All,np.array(DD_LDx_Crate['Cyc Last eta_tot'])*1e3,
                color=Colors[1],linestyle=LS[1], label=r"Double low $D_\times$")
    axs[1].plot(Rate_Dis_All,np.array(DD_LDx_Crate['Cyc Last eta_c_EC'])*1e3,
        color=Colors[1],linestyle=LS[1],label=r"Double low D$_\times$")
    axs[2].plot(Rate_Dis_All,np.array(DD_LDx_Crate['Cyc Last eta_c_e'])*1e3,
        color=Colors[1],linestyle=LS[1],label=r"Double low D$_\times$")

    axs[0].plot(Rate_Dis_All,np.array(DD_HDx_Crate['Cyc Last eta_tot'])*1e3,
                color=Colors[2],linestyle=LS[2], label=r"Double high $D_\times$")
    axs[1].plot(Rate_Dis_All,np.array(DD_HDx_Crate['Cyc Last eta_c_EC'])*1e3,
        color=Colors[2],linestyle=LS[2],label=r"Double high D$_\times$")
    axs[2].plot(Rate_Dis_All,np.array(DD_HDx_Crate['Cyc Last eta_c_e'])*1e3,
        color=Colors[2],linestyle=LS[2],label=r"Double high D$_\times$")
    axs[0].set_ylabel("$\eta$ [mV]")
    axs[0].set_xlabel("Discharge rate")
    axs[1].set_xlabel("Discharge rate")
    axs[2].set_xlabel("Discharge rate")
    axs[0].set_title("Total") 
    axs[1].set_title("EC") 
    axs[2].set_title(r"Li$^+$") 
    axs[0].legend(prop={'family':'Times New Roman','size':fs-2},
        loc='best',ncol=1,  #bbox_to_anchor=(1, -0.00),
        frameon=False,markerscale=0.5,borderpad=0.0,
        handlelength=1.0,columnspacing=1)
    if SaveFig == True:
        plt.savefig(BasicPath +  Target+ 
            f"Elely Overpotential last.png", dpi=1000)
        plt.savefig(BasicPath +  Target+ 
            f"Elely Overpotential last.svg")  
fs = 12;font = {'family' : 'Times New Roman','size': fs}
mpl.rc('font', **font)
Plot_Voldecom_last_Eley(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,SaveFig=True)
#Plot_Voldecom_mean_Eley(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,SaveFig=False)

In [ ]:
# old version - 3C rate: to plot only c_EC and c_Li+
LS = ['-','--','-','--'];
def Plot_Voldecom_1_Crate(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,Crate_index,SaveFig):
    fig, axs = plt.subplots(1,2, figsize=(9.3,3.2),tight_layout=True)
    sol = SD_Crate['Sol_All'][Crate_index]
    axs[0].plot(
        sol.cycles[0].steps[1]['Time [s]'].entries-sol.cycles[0].steps[1]['Time [s]'].entries[0], 
        sol.cycles[0].steps[1]["X-averaged battery concentration overpotential [V]"].entries,
        color=Colors[0],linestyle=LS[0],label=r"Single-High $D_\times$") 
    axs[1].plot(
        sol.cycles[0].steps[1]['Time [s]'].entries-sol.cycles[0].steps[1]['Time [s]'].entries[0], 
        sol.cycles[0].steps[1]["X-averaged EC concentration overpotential [V]"].entries,
        color=Colors[0],linestyle=LS[0],label=r"Single-High $D_\times$") 

    sol = DD_LDx_Crate['Sol_All'][Crate_index]
    axs[0].plot(
        sol.cycles[0].steps[1]['Time [s]'].entries-sol.cycles[0].steps[1]['Time [s]'].entries[0], 
        sol.cycles[0].steps[1]["X-averaged battery concentration overpotential [V]"].entries,
        color=Colors[3],linestyle=LS[3],label=r"Double-Low $D_\times$") 
    axs[1].plot(
        sol.cycles[0].steps[1]['Time [s]'].entries-sol.cycles[0].steps[1]['Time [s]'].entries[0], 
        sol.cycles[0].steps[1]["X-averaged EC concentration overpotential [V]"].entries,
        color=Colors[3],linestyle=LS[3],label=r"Double-Low $D_\times$") 

    sol = DD_HDx_Crate['Sol_All'][Crate_index]
    axs[0].plot(
        sol.cycles[0].steps[1]['Time [s]'].entries-sol.cycles[0].steps[1]['Time [s]'].entries[0], 
        sol.cycles[0].steps[1]["X-averaged battery concentration overpotential [V]"].entries,
        color=Colors[2],linestyle=LS[2],label=r"Double-High $D_\times$") 
    axs[1].plot(
        sol.cycles[0].steps[1]['Time [s]'].entries-sol.cycles[0].steps[1]['Time [s]'].entries[0], 
        sol.cycles[0].steps[1]["X-averaged EC concentration overpotential [V]"].entries,
        color=Colors[2],linestyle=LS[2],label=r"Double-High $D_\times$") 

    axs[0].set_ylabel("Potential [V]",fontsize=fs)
    axs[1].set_xlabel("Time [s]",fontsize=fs)
    axs[0].set_xlabel("Time [s]",fontsize=fs)
    fig.suptitle(f"EC and Li+ overpotential - {Rate_Dis_All[Crate_index]}C Discharge", fontsize=fs+1)
    if SaveFig == True:
        plt.savefig(BasicPath +  Target+ 
            f"EC and Li+ Overpotential {Rate_Dis_All[Crate_index]}C discharge.png", dpi=1000)
        plt.savefig(BasicPath +  Target+ 
            f"EC and Li+ Overpotential {Rate_Dis_All[Crate_index]}C discharge.svg")  
Plot_Voldecom_1_Crate(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,Crate_index=-1,SaveFig=False)
Plot_Voldecom_1_Crate(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,Crate_index=0,SaveFig=False)
Plot_Voldecom_1_Crate(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,Crate_index=5,SaveFig=False)

In [ ]:
# compare conccnetration
def Plot_Concentration_1_Crate(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,
    x_loc,var_elely,Title_keys,Crate_index,SaveFig):
    sol_Single_High =  SD_Crate['Sol_All'][Crate_index]
    time_Single_High = (sol_Single_High.cycles[0].steps[1]['Time [s]'].entries-sol_Single_High.cycles[0].steps[1]['Time [s]'].entries[0]).tolist()
    sol_Double_High = DD_HDx_Crate['Sol_All'][Crate_index]
    time_Double_High = (sol_Double_High.cycles[0].steps[1]['Time [s]'].entries-sol_Double_High.cycles[0].steps[1]['Time [s]'].entries[0]).tolist()
    sol_Double_Low = DD_LDx_Crate['Sol_All'][Crate_index]
    time_Double_Low = (sol_Double_Low.cycles[0].steps[1]['Time [s]'].entries-sol_Double_Low.cycles[0].steps[1]['Time [s]'].entries[0]).tolist()

    t_select = min(time_Single_High[-2],time_Double_High[-2],time_Double_Low[-2])
    i_Single_High = FindClose(time_Single_High,t_select)
    i_Double_High = FindClose(time_Double_High,t_select)

    i_Double_Low = FindClose(time_Double_Low,t_select)

    print(f'Time selected is for {Rate_Dis_All[Crate_index]}C is {time_Single_High[i_Single_High]} and {time_Double_High[i_Double_High]},{time_Double_Low[i_Double_Low]} s')
    Y_0p = np.arange(-1,10,0.1)
    fig, axs = plt.subplots(1,3, figsize=(18/2.54,4.5/2.54), ) #tight_layout=True
    for i in range(3):
        x_Single_High =   sol_Single_High.cycles[0].steps[1][x_loc[i]].entries[:,0]
        y_Single_High =   sol_Single_High.cycles[0].steps[1][var_elely[i]].entries[:,i_Single_High]

        x_Double_High =   sol_Double_High.cycles[0].steps[1][x_loc[i]].entries[:,0]
        y_Double_High =   sol_Double_High.cycles[0].steps[1][var_elely[i]].entries[:,i_Double_High]
        x_Double_Low =   sol_Double_Low.cycles[0].steps[1][x_loc[i]].entries[:,0]
        y_Double_Low =   sol_Double_Low.cycles[0].steps[1][var_elely[i]].entries[:,i_Double_Low]
        if var_elely[i].__contains__("[mol.m-3]"): 
            axs[i].plot(x_Single_High*1e6, y_Single_High/1e3, color=Colors[0],linestyle=LS[0],label=r"Single-High D$_\times$") 
            axs[i].plot(x_Double_Low*1e6, y_Double_Low/1e3, color=Colors[1],linestyle=LS[1],label=r"Double-Low D$_\times$") 
            axs[i].plot(x_Double_High*1e6, y_Double_High/1e3, color=Colors[2],linestyle=LS[0],label=r"Double-High D$_\times$") 
            #axs[i].plot( 85.2 * np.ones_like(Y_0p),Y_0p,"k-",linewidth=0.5,)
            #axs[i].plot((85.2+12 )* np.ones_like(Y_0p),Y_0p, "k-",linewidth=0.5,)
        else:
            axs[i].plot(x_Single_High*1e6, y_Single_High, color=Colors[0],linestyle=LS[0],label=r"Single-High $D_\times$") 
            axs[i].plot(x_Double_Low*1e6, y_Double_Low, color=Colors[1],linestyle=LS[1],label=r"Double-Low $D_\times$") 
            axs[i].plot(x_Double_High*1e6, y_Double_High, color=Colors[2],linestyle=LS[0],label=r"Double-High $D_\times$") 

        axs[i].set_xlabel(r"x [$\mu$m]",   fontdict={'family':'Times New Roman','size':fs})
        axs[i].set_title(Title_keys[i]) 
        axs[i].ticklabel_format( 
            axis='x', style='sci',
            scilimits=[-1e3,1e3], useOffset=None, 
            useLocale=None, useMathText=None)
    axs[0].set_ylabel("c [M]")
    axs[0].legend(prop={'family':'Times New Roman','size':fs-3},loc='best',frameon=False)  
    #axs[0].set_ylim([-0.1,4.1])
    #axs[1].set_ylim([1.7,4.8])
    #axs[2].set_ylim([5.5,7.6])
    # fig.suptitle(f"3C Discharge, t={round(t_select)} s", fontsize=fs+1)
    if SaveFig == True:
        plt.savefig(BasicPath +  Target+ 
            f"{Rate_Dis_All[Crate_index]}C Discharge, t={round(t_select)} s concentration profile.png", dpi=1000)
        plt.savefig(BasicPath +  Target+ 
            f"{Rate_Dis_All[Crate_index]}C Discharge, t={round(t_select)} s concentration profile.svg")  
x_loc = ["x [m]","x [m]","x [m]","x [m]","x [m]","x [m]"]
var_elely = [
    "Electrolyte concentration [mol.m-3]",
    "EC concentration [mol.m-3]",
    "EC:EMC wt%",]  # "c(EMC) [mol.m-3]",] 
var_LJP = [
    "dLJP_dce",
    "dLJP_dcEC",
    "Electrolyte concentration [mol.m-3]",] 
Title_LJP = [
    "dLJP_dce",
    "dLJP_dcEC",
    r"Li$^+$",]
Title_keys = [
    r"Li$^+$",
    "EC",
    "EC:EMC wt%",]
Plot_Concentration_1_Crate(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,
    x_loc,var_elely,Title_keys,Crate_index=5,SaveFig=False)
Plot_Concentration_1_Crate(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,
    x_loc,var_LJP,Title_LJP,Crate_index=5,SaveFig=False)
Plot_Concentration_1_Crate(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,
    x_loc,var_elely,Title_keys,Crate_index=0,SaveFig=False)

In [ ]:
# compare electrolyte properties
def Plot_EleyPro_1_Crate(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,
    x_loc,var_elely,Title_keys,Crate_index,Ylim,SaveFig):

    sol_Single_High =  SD_Crate['Sol_All'][Crate_index]
    time_Single_High = (sol_Single_High.cycles[0].steps[1]['Time [s]'].entries-sol_Single_High.cycles[0].steps[1]['Time [s]'].entries[0]).tolist()
    sol_Double_High = DD_HDx_Crate['Sol_All'][Crate_index]
    time_Double_High = (sol_Double_High.cycles[0].steps[1]['Time [s]'].entries-sol_Double_High.cycles[0].steps[1]['Time [s]'].entries[0]).tolist()
    sol_Double_Low = DD_LDx_Crate['Sol_All'][Crate_index]
    time_Double_Low = (sol_Double_Low.cycles[0].steps[1]['Time [s]'].entries-sol_Double_Low.cycles[0].steps[1]['Time [s]'].entries[0]).tolist()

    t_select = min(time_Single_High[-2],time_Double_High[-2],time_Double_Low[-2])
    i_Single_High = FindClose(time_Single_High,t_select)
    i_Double_High = FindClose(time_Double_High,t_select)

    i_Double_Low = FindClose(time_Double_Low,t_select)

    print(f'Time selected is for {Rate_Dis_All[Crate_index]}C is {time_Single_High[i_Single_High]} and {time_Double_High[i_Double_High]},{time_Double_Low[i_Double_Low]} s')
    
    Y_0p = np.arange(-1e-10,3,1e-1)
    fig, axs = plt.subplots(1,3, figsize=(19/2.54,6/2.54),tight_layout=True) # 
    for i in range(3):
        x_Single_High =   sol_Single_High.cycles[0].steps[1][x_loc[i]].entries[:,0]
        y_Single_High =   sol_Single_High.cycles[0].steps[1][var_elely[i]].entries[:,i_Single_High]

        x_Double_High =   sol_Double_High.cycles[0].steps[1][x_loc[i]].entries[:,0]
        y_Double_High =   sol_Double_High.cycles[0].steps[1][var_elely[i]].entries[:,i_Double_High]
        x_Double_Low =   sol_Double_Low.cycles[0].steps[1][x_loc[i]].entries[:,0]
        y_Double_Low =   sol_Double_Low.cycles[0].steps[1][var_elely[i]].entries[:,i_Double_Low]
        axs[i].plot( 85.2 * np.ones_like(Y_0p),Y_0p,"k-",linewidth=0.5,)
        axs[i].plot((85.2+12 )* np.ones_like(Y_0p),Y_0p, "k-",linewidth=0.5,)
        axs[i].plot(x_Single_High*1e6, y_Single_High, color=Colors[0],linestyle=LS[0],label=r"Single-High $D_\times$") 
        axs[i].plot(x_Double_Low*1e6, y_Double_Low, color=Colors[1],linestyle=LS[1],label=r"Double-Low $D_\times$") 
        axs[i].plot(x_Double_High*1e6, y_Double_High, color=Colors[2],linestyle=LS[0],label=r"Double-High $D_\times$") 
        axs[i].set_xlabel(r"x [$\mu$m]",   fontdict={'family':'Times New Roman','size':fs})
        #axs[i].set_title(Title_keys[i]) 
        axs[i].ticklabel_format( 
            axis='x', style='sci',
            scilimits=[-1e3,1e3], useOffset=None, 
            useLocale=None, useMathText=None)
        axs[i].set_ylim(Ylim[i])
    axs[0].set_ylabel("$\kappa_\mathrm{e}$ [S/m]")
    axs[1].set_ylabel("D$_\mathrm{e}$ [m$^\mathrm{2}$/s]") # 
    axs[2].set_ylabel("$\mathit{t}_\mathrm{0}^\mathrm{+}$")
    """ axs[2].legend(prop={'family':'Times New Roman','size':fs-2},
    loc='best',ncol=1,  #bbox_to_anchor=(1, -0.00),
    frameon=False,markerscale=0.5,borderpad=0.0,
    handlelength=1.0,columnspacing=1) """
    if SaveFig == True:
        plt.savefig(BasicPath +  Target+ 
            f"Fig_S2_{Rate_Dis_All[Crate_index]}C Discharge, t={round(t_select)} s electrolyte properties.png", dpi=1000)
        plt.savefig(BasicPath +  Target+ 
            f"Fig_S2_{Rate_Dis_All[Crate_index]}C Discharge, t={round(t_select)} s electrolyte properties.svg") 
x_loc = ["x [m]","x [m]","x [m]","x [m]","x [m]","x [m]"]
var_elely = [
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
    "Cation transference number",]
Title_keys = [
    r"Li$^+$",
    "EC",
    "EMC",]
fs = 12
font = {'family' : 'Times New Roman','size'   : fs}
mpl.rc('font', **font)
# Rate_Dis_All = [ 1, 1.75,2,2.25, 2.5, 3, 3.25, 3.5, 3.75, 4 ,4.2,4.4, 4.5 ];
Ylim = [ [-0.1,1.25] ,  [1e-11,5.4e-10] , [0.08,0.6] ] # for 4.5C
for i in range(len(Rate_Dis_All)):
    Plot_EleyPro_1_Crate(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,
        x_loc,var_elely,Title_keys,Crate_index=i,Ylim=Ylim,SaveFig=True)
#Plot_EleyPro_1_Crate(SD_Crate,DD_LDx_Crate,DD_HDx_Crate,x_loc,var_elely,Title_keys,Crate_index=0,SaveFig=False)
